In [1]:
import codecs
import re
import os
import pickle

inverts_index = {}
bookz = os.listdir('./books') 
l = len(bookz)
matrix = []
terms = []

for b in bookz:
  file_ = open(os.path.join('./books', b), "r")
  f = file_.read()
  w = re.findall("[a-z]+[-']?[a-z]+", f.lower())
  file_.close()
  for el in w:
    if el not in inverts_index:
      inverts_index[el] = [b]
    else:
      inverts_index[el].append(b)

    if el not in terms:
      terms.append(el)
      zeros = [0] * l
      zeros[bookz.index(b)] = 1
      matrix.append(zeros)
    else:
      matrix[terms.index(el)][bookz.index(b)] = 1

In [2]:
import sys
print('Size of inverts index:   ', sys.getsizeof(inverts_index))
print('Size of incidence matrix: ', sys.getsizeof(matrix + terms + bookz))

Size of inverts index:    2621552
Size of incidence matrix:  701176


In [3]:
i = 0
for key, value in inverts_index.items():
  i += 1
  print(key,': ', value)
  if i == 400:
    break

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




charm :  ['The Pathfinder, or The Inland Sea.txt', 'The Pathfinder, or The Inland Sea.txt', 'The Pathfinder, or The Inland Sea.txt', 'The Pathfinder, or The Inland Sea.txt', 'Rob Roy.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'Sister Carrie.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt', 'The House of the Seven Gables.txt

In [4]:
i = 0
print('\t\t', end='', flush=True) 
for b in bookz:
  m = ' ' * (37-len(b))
  print(b,m,'|', end='', flush=True) 
print()

for index, value in enumerate(matrix):
  i += 1
  if len(terms[index]) >= 7:
    print(terms[index],'\t', end='', flush=True)
    
  else:
    print(terms[index],'\t\t', end='', flush=True)
  for k in value:
    print(k,'\t\t\t\t\t', end='', flush=True)
  print()
  if i == 10:
    break

		The Pathfinder, or The Inland Sea.txt  |Rob Roy.txt                            |Sister Carrie.txt                      |The House of the Seven Gables.txt      |Montezuma's Daughter.txt               |The White Company.txt                  |At the Earth's Core.txt                |The Prairie.txt                        |Gone with the Wind.txt                 |The Antiquary.txt                      |The Last of the Mohicans - A narrative of 1757.txt  |Oliver Twist.txt                       |
chapter 	1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					
the 		1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					
turf 		1 					1 					0 					0 					1 					1 					0 					0 					1 					1 					1 					1 					
shall 		1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					
be 		1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					1 					
my 		1 					1 					1 					1 					1 					1 		

In [5]:
query = 'chapter AND NOT turf OR fragrant'
comands = ['NOT', 'AND', 'OR']
query = query.split()

In [6]:
result = ''
curr = ''

def fOR(a,b):
  if a == 1:
    return 1
  elif b == 1:
    return 1
  else:
    return 0

def fAND(a,b):
  if a == 1 and b == 1:
    return 1
  else:
    return 0   

for index, item in enumerate(query):
  if item not in comands:
    curr = matrix[terms.index(item)]
    if index != 0:
      if query[index-1] == 'NOT':
        curr = [(x+1)%2 for x in curr]
      if result == '':
        result = curr
      if query[index-1] == 'AND' or (query[index-2] == 'AND' and query[index-1] == 'NOT'):
        result = [fAND(result[x],curr[x]) for x in range(l)]
      if query[index-1] == 'OR' or (query[index-2] == 'OR' and query[index-1] == 'NOT'):
        result = [fOR(result[x],curr[x]) for x in range(l)]

print(result)

for index, b in enumerate(bookz):
  if result[index] == 1:
    print(b)

[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0]
The Pathfinder, or The Inland Sea.txt
Rob Roy.txt
Sister Carrie.txt
The House of the Seven Gables.txt
At the Earth's Core.txt
The Prairie.txt
Gone with the Wind.txt
The Last of the Mohicans - A narrative of 1757.txt


In [7]:
result = ''
curr = ''

for index, item in enumerate(query):
  if item not in comands:
    curr = inverts_index[item]
    if index != 0:
      if query[index-1] == 'NOT':
        c = []
        for b in bookz:
          if b not in curr:
            c.append(b)
        curr = c

      if result == '':
        result = curr

      if query[index-1] == 'AND' or (query[index-2] == 'AND' and query[index-1] == 'NOT'):
        c = []
        for b in result:
          if b in curr:
            c.append(b)
        result = c

      if query[index-1] == 'OR' or (query[index-2] == 'OR' and query[index-1] == 'NOT'):
        result = list(set(result+curr))

for b in result:
    print(b)

The Last of the Mohicans - A narrative of 1757.txt
At the Earth's Core.txt
Sister Carrie.txt
The Prairie.txt
The House of the Seven Gables.txt
The Pathfinder, or The Inland Sea.txt
Rob Roy.txt
Gone with the Wind.txt
